<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages needed**

In [1]:
!pip install gekko
!pip install ipynb


     |████████████████████████████████| 10.8MB 5.0MB/s 


**Importing libraries needed**

In [2]:
import numpy as np
import time
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *
#Initialize Model
m = GEKKO(remote=True) 
#Set global options
m.options.solver = 3
m.options.IMODE = 3 #steady state optimization


Mounted at /content/drive


In [3]:
#Upload distances from each hexagon to other
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Distance_TN.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN.csv")


**SAWMILL CLASS**

In [4]:
m = GEKKO(remote=True)
class Inventory:
  def __init__(self, Av, Dist):
    self.Available = []
    self.Available.append(Av) #Dataframe that contains Available wood Inventory at diferent time t
    self.Dist = Dist #Matrix Distance from any cell to another cell
    self.Mills = [] #Dataframe that contains Id, typewood, type_product, and demand of mills
    self.Dist_Mill = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Available_Mill = [] #Dataframe that contains the Available Inventory close the mill to be harvested
    self.Harvested = []
    self.Residues = []
    self.Growth = 0
    self.WoodType = 0
    self.ProdType = 0

  def Harvest_Mill(self,t, j):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    Id1 =self.Mills.loc[j,('Unique_ID')]
    a =self.Cell_Less(Id1,0,20)
    self.Dist_Mill = a.loc[:, ('DEST','DIST')]
    Num = len(self.Dist_Mill)
    
    H = m.Array(m.Var,(Num),value = 0)
    Dem = self.Mills.loc[j,('Demand')]
    self.Available_Mill = self.Disp_Radio( Id1, t, 20)
    
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Mill[i]
      
    prod0 = m.Intermediate(m.sum([self.Dist_Mill.loc[i,('DIST')]*H[i] for i in range(Num)]))
    
    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=False)
    a = self.Dist_Mill
    flat = [item for l in H for item in l]
    a['Harvested'] = flat 
    ans = a
    
    #Storing Harvested 
    for j in range(len(ans)):                                                                         
      if not(self.Harvested[0][self.Harvested[0]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested[0]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested[0].loc[a2,("Harvested")] = self.Harvested[0].loc[a2,("Harvested")] + ans.loc[j,('Harvested')]  
    print('Harvested[0]\n',self.Harvested[0])
    print('Sum Harvested\n',sum(self.Harvested[0].Harvested))

    #Discounting Harvested from Inventory
    for j in range(len(ans)):                                                                         
      if not(self.Available[0][self.Available[0]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Available[0]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Available[0].loc[a2,("SumOfTotal")] = self.Available[0].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')]  
  
  def Harvest_All_Mills(self,k):
    for i in range(k):
      print('Cosechando Sawmill:',i)
      Hardwood_Sawlog.Harvest_Mill(0,i)
    
  
  def Inventory_Growth(self,t,rate = 0):
    if (rate == 0):
      r = self.Growth
    else:
      r = rate

    print('Available:',self.Available[t].loc[:,("SumOfTotal")])
    self.Available[t].loc[:,("SumOfTotal")] = self.Available[t].loc[:,("SumOfTotal")]*(1+r/100)
    print('Available:',self.Available[t].loc[:,("SumOfTotal")])


  def Cell_Less(self, Id_Mill, Time, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of cells with Distance and Availability
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):                                                                         
      if self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))

    Ans["AVAIL"] = Disp 
    return(Ans)

  def Disp_Radio(self, Id_Mill, Time, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of Inventory Available
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    for j in range(len(Ans)):                                                                         
      if self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))

    return(Disp)
  

  def Num_Cell(self, Id_Mill, Radio=10):
    #Given a Radio and a Mill ID this function returns cell among the radio distance
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)]
    #print("Numero de celdas ",len(Ans))
    return(len(Ans))
    


**UPLOADING FOREST INVENTORY DATA**

**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [ ]:
# Main Program
start_time = time.time()
Hardwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Sawlog.csv")

Hardwood_Sawlog = Inventory(Hardwood_Sawlog_Data,Dist_Data)
v1 = Hardwood_Sawlog.Available[0]["Unique_ID"]==54549
v2 =int(Hardwood_Sawlog.Available[0].loc[v1,("SumOfTotal")])
print('Before harvesting',v2)
r1 = len(Hardwood_Sawlog.Available[0])
Id_li = list(Hardwood_Sawlog.Available[0].loc[:,("Unique_ID")])
Ha_li =[0]*r1
Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
Hardwood_Sawlog.Harvested.append(pd.DataFrame(Data))
#Hardwood_Sawlog.Harvested[0].insert(0,"Unique_ID", l]
#Hardwood_Sawlog.Harvested[0].rename(columns=['Unique_ID','Harvested'])
#print("DataFrame Harvested\n",Hardwood_Sawlog.Harvested[0])
Num_Mills = 20
ma = Dist_Data['ORIG'].unique()
print('Ma', ma[0])

for k in range(Num_Mills):
  Mills_Data.loc[k,('Unique_ID')] = ma[k]
  #Mills_Data.loc[k,('Demand')] = 10000000

Hardwood_Sawlog.Mills=Mills_Data
Hardwood_Sawlog.Harvest_All_Mills(Num_Mills)
  
print('AFTER HARVEST',Hardwood_Sawlog.Available[0].loc[v1,("SumOfTotal")])
#compression_opts = dict(method='zip',
#                        archive_name='out.csv')  
Hardwood_Sawlog.Harvested[0].to_csv('/content/drive/My Drive/Colab Notebooks/out.cvs')
Mills_Data.to_csv('/content/drive/My Drive/Colab Notebooks/Mills_TN_CF.cvs')

print ("My program took", time.time() - start_time, "to run")

Before harvesting 2287844
Ma 54549
Cosechando Sawmill: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Harvested[0]
       Unique_ID  Harvested
0         74031        0.0
1         75559        0.0
2         86608        0.0
3         68554        0.0
4         68082        0.0
...         ...        ...
7758      76917        0.0
7759      61910        0.0
7760      76697        0.0
7761      75323        0.0
7762      54749  2268199.0

[7763 rows x 2 columns]
Sum Harvested
 9999999.908760365
Cosechando Sawmill: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Harvested[0]
       Unique_ID  Harvested
0         74031        0.0
1         75559        0.0
2         86608        0.0
3         68554        0.0
4         68082        0.0
...         ...        ...
7758      76917        0.0
7759      61910        0.0
7760      76697        0.0
7761      75323        0.0
7762      54749  2268199.0

[7763 rows x 2 columns]
Sum Harvested
 19999999.825660363
Cosechando Sawmill: 2


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Harvested[0]
       Unique_ID     Harvested
0         74031  0.000000e+00
1         75559  0.000000e+00
2         86608  0.000000e+00
3         68554  0.000000e+00
4         68082  0.000000e+00
...         ...           ...
7758      76917  0.000000e+00
7759      61910  2.334858e-08
7760      76697  0.000000e+00
7761      75323  0.000000e+00
7762      54749  2.268199e+06

[7763 rows x 2 columns]
Sum Harvested
 359999996.8412161
Cosechando Sawmill: 36


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Harvested[0]
       Unique_ID     Harvested
0         74031  0.000000e+00
1         75559  0.000000e+00
2         86608  0.000000e+00
3         68554  0.000000e+00
4         68082  0.000000e+00
...         ...           ...
7758      76917  0.000000e+00
7759      61910  2.334858e-08
7760      76697  0.000000e+00
7761      75323  0.000000e+00
7762      54749  2.268199e+06

[7763 rows x 2 columns]
Sum Harvested
 369999996.75741595
Cosechando Sawmill: 37


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
